In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)

from src.extractor_utils import (predict, getPrediction,features_saving,features_loading)

from utils.utils import (create_submission ,metrics2csv, results2csv)

In [2]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

4 GPU's available:

cpu_count: 24


In [3]:
use_gpu = torch.cuda.is_available()
use_DataParalel= True 
use_CPU= False        

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))
    else:
        print('Using only one GPU') 
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using DataParalel in all 4 GPUS


In [4]:
if use_gpu:
    !free -h
    !nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         66G         67G        3.7G         86G        148G
Swap:            0B          0B          0B
Sat Nov 25 23:26:33 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   42C    P0    71W / 149W |   7028MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0


# Choose dataset 

In [5]:
path2train = "/mnt/home/e209440/data/train" 
path2test = "/mnt/home/e209440/data/test" 

# Images transformation

In [6]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}        

# Creating  Dataset & Dataloader

In [7]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [8]:
dsets = {
    'train': KaggleSafeDriverDataset(path2train, transforms=data_transforms['train'],use_only=use_only),
    'valid': KaggleSafeDriverDataset(path2train, transforms=data_transforms['valid'],use_only=use_only, is_val=True, val_size=0.2),
    'test':  KaggleSafeDriverDataset(path2test, transforms=data_transforms['valid'],use_only=use_only, is_test=True),
}

In [9]:
dset_loaders = _create_dataLoader(dsets, batch_size, pin_memory=False, use_shuffle= True)

In [10]:
dset_sizes = {x: len(dsets[x]) for x in ['train','valid', 'test']} 
dset_classes = len(dsets['train'].y)
dset_classes, dset_sizes

# Dataset have much more samples than datatrain ***It comes from the test.zip****

(17940, {'test': 79726, 'train': 17940, 'valid': 4484})

# Checking GPU usage

In [11]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         66G         67G        3.7G         86G        148G
Swap:            0B          0B          0B
Sat Nov 25 23:26:50 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   42C    P0    71W / 149W |   7028MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

# Choosing Model

In [16]:
use_resnet = True
use_inception = False
use_denseNet = False

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
if use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
if use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()

Using ResNet model


In [19]:
if use_gpu:
    if use_DataParalel:
        print("Using all GPU's ")
        convnet = torch.nn.DataParallel(model.mrnc) #device_ids=[1,3]
        convnet = convnet.cuda()
        densenet = torch.nn.DataParallel(model.mrnd) #device_ids=[1,3]
        densenet = densenet.cuda()
    else:
        print('Using GPU')# {}'.format(device_id))
        model.cuda()
        convnet = model.mrnc
        densenet = model.mrnd
else:
    print("Using CPU's")
    convnet = model.mrnc
    densenet = model.mrnd

Using CPU's


In [58]:
#print(model)

In [20]:
inp = torch.autograd.Variable(torch.randn(1, 3, 300,300).cuda())
out=convnet(inp)
out.data.shape

torch.Size([1, 6400])

In [21]:
#extract features from images
convOutput_train = predict(dset_loaders['train'], convnet,use_gpu=use_gpu)
convOutput_valid = predict(dset_loaders['valid'], convnet,use_gpu=use_gpu)
convOutput_test = predict(dset_loaders['test'], convnet,use_gpu=use_gpu)

predict: 560/560 ok
predict: 140/140 ok
predict: 2491/2491 ok


In [28]:
print(convOutput_train['true'].size(), convOutput_train['pred'].size())
print(convOutput_valid['true'].size(), convOutput_valid['pred'].size())
print(convOutput_test['true'].size(), convOutput_test['pred'].size())

torch.Size([17940]) torch.Size([17940, 6400])
torch.Size([4484]) torch.Size([4484, 6400])
torch.Size([79726]) torch.Size([79726, 6400])


In [29]:
print(convOutput_train['true'].type(), convOutput_train['pred'].type())

torch.LongTensor torch.FloatTensor


# Saving features

In [30]:
path2features= "/mnt/home/e209440/state-farm-distracted-driver-detection/features/" 
#path2features= "/mnt/home/r120084/state-farm-distracted-driver-detection/features/" 

In [31]:
sav_feats= {
    'train': (convOutput_train['pred'], convOutput_train['true'],model_name),
    'valid': (convOutput_valid['pred'], convOutput_valid['true'],model_name),
    'test': (convOutput_test['pred'], convOutput_test['true'],model_name)

}

In [32]:
sav_feats['train'][2]

'ResNet'

In [33]:
features_saving(path2features,sav_feats)


Saving ResNet train features
Saved in:/mnt/home/e209440/state-farm-distracted-driver-detection/features/train/ResNetFeatures.npz

Saving ResNet valid features
Saved in:/mnt/home/e209440/state-farm-distracted-driver-detection/features/valid/ResNetFeatures.npz

Saving ResNet test features
Saved in:/mnt/home/e209440/state-farm-distracted-driver-detection/features/test/ResNetFeatures.npz


# Loading features

In [34]:
model_name='ResNet'

In [35]:
load_feat= features_loading(path2features,model_name)

Loaded features with shapes: 


train:
pred torch.Size([17940, 6400]), true torch.Size([17940])

valid:
pred torch.Size([4484, 6400]), true torch.Size([4484])

test:
pred torch.Size([79726, 6400]), true torch.Size([79726])


In [ ]:
load_feat['train'][0].shape,load_feat['train'][1].shape

In [ ]:
conv_dset ={
    'train': TensorDataset(load_feat['train'][0], load_feat['train'][1]),
    'valid': TensorDataset(load_feat['valid'][0], load_feat['valid'][1]),
    'test': TensorDataset(load_feat['test'][0], load_feat['test'][1])
} 

In [ ]:
dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [ ]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

In [ ]:
#path2saveModel = '/mnt/home/r120084/project/models/distractdriver'
#path2saveModel = '/mnt/home/e209440/models/ResNetDistractDriver' 

savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=1)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
num_epochs = 10

optimizer =  optim.Adam(densenet.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

params = {'model' : densenet, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
    'callbacks': [savebest, ptt.AccuracyMetric(), ptt.PrintCallback()] #ptt.PlotCallback(),
}

In [ ]:
trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,**params)

In [ ]:
trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

# Loading best epoch

In [ ]:
trainer.load_state(path2saveModel)
densenet = trainer.model

In [ ]:
!free -h
!nvidia-smi

In [ ]:
metrics2csv(trainer, 'ResNet18')

In [ ]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

In [ ]:
train_eval, valid_eval

In [ ]:
# need to create a dataloader without shuffle to compare the mismatch
dset_loaders_wshuffle = _create_dataLoader(dsets, batch_size, pin_memory=False, use_shuffle= False)

In [ ]:
#we need to use all dloader, because this one has use_shuffle false
result_train = predict(dset_loaders_wshuffle['train'], model, use_gpu=use_gpu)
result_valid = predict(dset_loaders_wshuffle['valid'], model, use_gpu=use_gpu)
result_test = predict(dset_loaders_wshuffle['test'], model, use_gpu=use_gpu)

In [ ]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [ ]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [ ]:
print('Train: ', correct_train, '/', len(dsets['train']) )
print('Valid: ', correct_valid, '/', len(dsets['valid']) )

In [ ]:
visualize_predictions(dsets['valid'], result_valid, correct_pred = True)

In [ ]:
predictions_out = {'train': result_train, 'valid': result_valid}

In [ ]:
results2csv(predictions_out, 'Resnet18')

In [ ]:
#move this to Data Visualization
#visualize_predictions(dsets['valid'], result_valid, correct_pred = False)

In [ ]:
#plot_cm_train_valid(result_train,result_valid)

In [ ]:
#move this to Data Visualization
#plot_layers_weight(dsets,img_width=img_width, img_height=img_height,conv_model = model.mrnc,use_gpu=use_gpu)

# Make submission of the Test set

In [ ]:
create_submission(result_test, 'distracted_driver')